In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

In [2]:
# Load datasets
train_df = pd.read_csv('training.data', header=None, na_values='?')
test_df = pd.read_csv('test.data', header=None, na_values='?')

# Define column names
columns = [f'A{i}' for i in range(1, 16)] + ['A16']
train_df.columns = columns
test_df.columns = columns

# Identify categorical and continuous attributes
categorical_attrs = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']
continuous_attrs = ['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

# Handle missing values
def handle_missing_values(train_data, test_data):
    medians = {}
    for attr in train_data.columns[:-1]:
        if attr in categorical_attrs:
            valid_values = train_data[attr].dropna().sort_values()
            medians[attr] = valid_values.iloc[len(valid_values) // 2]
        else:
            medians[attr] = train_data[attr].median()
    for attr in train_data.columns[:-1]:
        train_data[attr] = train_data[attr].fillna(medians[attr])
        test_data[attr] = test_data[attr].fillna(medians[attr])
    return train_data, test_data

train_df, test_df = handle_missing_values(train_df, test_df)

# Convert to numpy arrays for efficiency
train_data = train_df.to_numpy()
test_data = test_df.to_numpy()
attribute_indices = {col: i for i, col in enumerate(columns)}

In [3]:
# Entropy calculation
def entropy(labels):
    if len(labels) == 0:
        return 0
    counts = np.bincount(np.where(labels == '+', 1, 0))
    probs = counts / len(labels)
    return -np.sum([p * math.log2(p) for p in probs if p > 0])

# Plurality value
def plurality_value(labels):
    if len(labels) == 0:
        return '+'
    return '+' if np.sum(labels == '+') >= np.sum(labels == '-') else '-'

# Accuracy calculation
def accuracy(true_labels, pred_labels):
    return np.mean(true_labels == pred_labels)

# F1 Score calculation
def f1_score(true_labels, pred_labels):
    tp = np.sum((true_labels == '+') & (pred_labels == '+'))
    fp = np.sum((true_labels == '-') & (pred_labels == '+'))
    fn = np.sum((true_labels == '+') & (pred_labels == '-'))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

In [4]:
# Precompute unique values for efficiency
def precompute_values(data, attr_idx):
    return np.unique(data[:, attr_idx])

# Information Gain with precomputed values
def information_gain(data, attr_idx, label_idx):
    total_entropy = entropy(data[:, label_idx])
    values = precompute_values(data, attr_idx)
    weighted_entropy = 0
    for value in values:
        subset = data[data[:, attr_idx] == value]
        weighted_entropy += len(subset) / len(data) * entropy(subset[:, label_idx])
    return total_entropy - weighted_entropy

# Gain Ratio (C4.5)
def gain_ratio(data, attr_idx, label_idx):
    gain = information_gain(data, attr_idx, label_idx)
    values = precompute_values(data, attr_idx)
    intrinsic_value = -sum(len(data[data[:, attr_idx] == v]) / len(data) *
                           math.log2(len(data[data[:, attr_idx] == v]) / len(data))
                           for v in values)
    return gain / intrinsic_value if intrinsic_value > 0 else 0

# Gini Index (CART)
def gini_index(data, attr_idx, label_idx):
    values = precompute_values(data, attr_idx)
    weighted_gini = 0
    for value in values:
        subset = data[data[:, attr_idx] == value]
        if len(subset) == 0:
            continue
        counts = np.bincount(np.where(subset[:, label_idx] == '+', 1, 0), minlength=2)
        probs = counts / len(subset)
        gini = 1 - np.sum(probs ** 2)
        weighted_gini += len(subset) / len(data) * gini
    return weighted_gini

In [5]:
# Optimized best split for continuous attributes
def best_split(data, attr_idx, label_idx):
    values = np.sort(np.unique(data[:, attr_idx].astype(float)))
    if len(values) < 2:
        return -float('inf'), None
    midpoints = (values[:-1] + values[1:]) / 2
    best_gain = -float('inf')
    best_split_point = None
    labels = data[:, label_idx]
    total_entropy = entropy(labels)

    for midpoint in midpoints:
        left_mask = data[:, attr_idx].astype(float) <= midpoint
        left_labels = labels[left_mask]
        right_labels = labels[~left_mask]
        if len(left_labels) < 2 or len(right_labels) < 2:  # Minimum split size
            continue
        gain = total_entropy - (len(left_labels) / len(labels) * entropy(left_labels) +
                                len(right_labels) / len(labels) * entropy(right_labels))
        if gain > best_gain:
            best_gain = gain
            best_split_point = midpoint
    return best_gain if best_gain != -float('inf') else -float('inf'), best_split_point

In [6]:
class Node:
    def __init__(self, attribute=None, value=None, label=None, branches=None):
        self.attribute = attribute
        self.value = value
        self.label = label
        self.branches = branches or {}

In [7]:
def decision_tree_learning(data, attributes, parent_data, importance_func, max_depth=10, min_gain=0.01, depth=0):
    labels = data[:, -1]
    if len(data) < 2 or depth >= max_depth:
        return Node(label=plurality_value(labels))
    if np.all(labels == labels[0]):
        return Node(label=labels[0])
    if not attributes:
        return Node(label=plurality_value(labels))

    label_idx = attribute_indices['A16']
    if importance_func == gain_ratio:
        default_value = -float('inf')
        best_attr = max(attributes, key=lambda a: gain_ratio(data, attribute_indices[a], label_idx) if a in categorical_attrs
                        else best_split(data, attribute_indices[a], label_idx)[0])
        best_score = (gain_ratio(data, attribute_indices[best_attr], label_idx) if best_attr in categorical_attrs
                      else best_split(data, attribute_indices[best_attr], label_idx)[0])
    else:  # CART
        default_value = float('inf')
        best_attr = min(attributes, key=lambda a: gini_index(data, attribute_indices[a], label_idx) if a in categorical_attrs
                        else best_split(data, attribute_indices[a], label_idx)[0])
        best_score = (gini_index(data, attribute_indices[best_attr], label_idx) if best_attr in categorical_attrs
                      else best_split(data, attribute_indices[best_attr], label_idx)[0])

    if best_score <= min_gain or (best_attr in continuous_attrs and best_split(data, attribute_indices[best_attr], label_idx)[1] is None):
        return Node(label=plurality_value(labels))

    attr_idx = attribute_indices[best_attr]
    tree = Node(attribute=best_attr)
    if best_attr in continuous_attrs:
        gain, split_point = best_split(data, attr_idx, label_idx)
        tree.value = split_point
        left_mask = data[:, attr_idx].astype(float) <= split_point
        tree.branches['<='] = decision_tree_learning(data[left_mask], [a for a in attributes if a != best_attr],
                                                     data, importance_func, max_depth, min_gain, depth + 1)
        tree.branches['>'] = decision_tree_learning(data[~left_mask], [a for a in attributes if a != best_attr],
                                                    data, importance_func, max_depth, min_gain, depth + 1)
    else:
        values = precompute_values(data, attr_idx)
        for value in values:
            subset = data[data[:, attr_idx] == value]
            if len(subset) > 0:
                tree.branches[value] = decision_tree_learning(subset, [a for a in attributes if a != best_attr],
                                                              data, importance_func, max_depth, min_gain, depth + 1)

    return tree

In [8]:
def predict(tree, example):
    if tree.label is not None:
        return tree.label
    attr_idx = attribute_indices[tree.attribute]
    if tree.attribute in continuous_attrs:
        return predict(tree.branches['<='], example) if float(example[attr_idx]) <= tree.value else predict(tree.branches['>'], example)
    value = example[attr_idx]
    return predict(tree.branches[value], example) if value in tree.branches else plurality_value(train_data[:, -1])

In [9]:
def cross_validation(data, importance_func):
    fold_size = 55
    folds = [data[i:i + fold_size] for i in range(0, len(data), fold_size)]
    acc_scores, f1_scores = [], []

    for i in range(10):
        val_fold = folds[i]
        train_folds = np.concatenate([folds[j] for j in range(10) if j != i])
        tree = decision_tree_learning(train_folds, columns[:-1], train_folds, importance_func)
        true_labels = val_fold[:, -1]
        pred_labels = np.array([predict(tree, ex) for ex in val_fold])
        acc_scores.append(accuracy(true_labels, pred_labels))
        f1_scores.append(f1_score(true_labels, pred_labels))

    return np.mean(acc_scores), np.mean(f1_scores), tree

In [10]:
# Main execution//
print("Running C4.5...")
c4_5_acc, c4_5_f1, c4_5_tree = cross_validation(train_data, gain_ratio)
print(f"C4.5 Cross-Validation Accuracy: {c4_5_acc:.4f}")
print(f"C4.5 Cross-Validation F1 Score: {c4_5_f1:.4f}")

print("\nRunning CART...")
cart_acc, cart_f1, cart_tree = cross_validation(train_data, gini_index)
print(f"CART Cross-Validation Accuracy: {cart_acc:.4f}")
print(f"CART Cross-Validation F1 Score: {cart_f1:.4f}")

# Select best model based on F1 score
best_tree = c4_5_tree if c4_5_f1 > cart_f1 else cart_tree
best_model = "C4.5" if c4_5_f1 > cart_f1 else "CART"
test_true_labels = test_data[:, -1]
test_pred_labels = np.array([predict(best_tree, ex) for ex in test_data])
test_acc = accuracy(test_true_labels, test_pred_labels)
test_f1 = f1_score(test_true_labels, test_pred_labels)

print(f"\nBest Model: {best_model}")
print(f"Test Set Accuracy: {test_acc:.4f}")
print(f"Test Set F1 Score: {test_f1:.4f}")

# Comparison
print("\nComparison of C4.5 and CART:")
print(f"C4.5 - Accuracy: {c4_5_acc:.4f}, F1: {c4_5_f1:.4f}")
print(f"CART - Accuracy: {cart_acc:.4f}, F1: {cart_f1:.4f}")
print(f"Test Set - Accuracy: {test_acc:.4f}, F1: {test_f1:.4f} with {best_model}")

Running C4.5...
C4.5 Cross-Validation Accuracy: 0.8145
C4.5 Cross-Validation F1 Score: 0.7791

Running CART...
CART Cross-Validation Accuracy: 0.7636
CART Cross-Validation F1 Score: 0.7298

Best Model: C4.5
Test Set Accuracy: 0.8143
Test Set F1 Score: 0.7969

Comparison of C4.5 and CART:
C4.5 - Accuracy: 0.8145, F1: 0.7791
CART - Accuracy: 0.7636, F1: 0.7298
Test Set - Accuracy: 0.8143, F1: 0.7969 with C4.5
